In [ ]:
%matplotlib inline

In [ ]:
%run ../src/benchmark.py
%run ../src/plotting.py

In [ ]:
from framed import load_cbmodel
from framed.cobra.ensemble import load_ensemble
import pandas as pd
import matplotlib.pyplot as plt
from carveme.reconstruction.utils import load_media_db
import seaborn as sns
sns.set_style('white')

### Select organism

In [ ]:
SELECTED = 1

organisms = [
    'ecoli',
    'bsubtilis',
    'paeruginosa',
    'soneidensis',
    'mgenitalium'
]

organism = organisms[SELECTED]

### Load gene essentiality data

In [ ]:
if organism == 'ecoli':
    medium_tag = 'M9'
    df = pd.read_csv('../experimental/essentiality/ecoli/essential_glc_mm.csv', header=None)
    essential = set(df[0].apply(lambda x: 'G_' + x))
    non_essential = None
    
elif organism == 'bsubtilis':
    medium_tag = 'LB'
    df = pd.read_csv('../experimental/essentiality/bsubtilis/essential_rich_medium.tsv', sep='\t')
    df = df.fillna('')
    essential = {'G_' + row['bigg_id'] for _, row in df.iterrows()
                 if row['phenotype'] == 'essential' and row['bigg_id'] != ''}
    non_essential = {'G_' + row['bigg_id'] for _, row in df.iterrows()
                      if row['phenotype'] == 'non-essential' and row['bigg_id'] != ''}    

elif organism == 'mgenitalium':
    medium_tag = None
    df = pd.read_csv('../experimental/essentiality/mgenitalium/essentiality_rich_medium.tsv', sep='\t')
    essential = {'G_' + row['locus'] for _, row in df.iterrows() if row['essential'] == 'E'}
    non_essential = {'G_' + row['locus'] for _, row in df.iterrows() if row['essential'] == 'NE'}    

elif organism == 'paeruginosa':
    medium_tag = 'M9[succ]'
    df = pd.read_csv('../experimental/essentiality/paeruginosa/essential_MOPS_succ.tsv', sep='\t')
    essential = {'G_' + row['locus'] for _, row in df.iterrows() if row['essential'] == 'E'}
    non_essential = {'G_' + row['locus'] for _, row in df.iterrows() if row['essential'] == 'NE'}
    
elif organism == 'soneidensis':
    medium_tag = 'LB'
    df = pd.read_csv('../experimental/essentiality/soneidensis/essentiality_price2016.tsv', header=None)
    essential = set(df[0].apply(lambda x: 'G_' + x))
    non_essential = None    
    

### Define simulation conditions

In [ ]:
test = 'main'
#test = 'biomass'
#test = 'ensemble'

styles = None
voting_thresholds = None

In [ ]:
if test == 'biomass':
    styles = [(c, '-', True) for c in sns.color_palette('YlGnBu', n_colors=3)]
    
if test == 'ensemble':
    c = sns.color_palette('YlGnBu', n_colors=4)
    styles = [(c[0], '-', True), (c[1], '-', False), (c[2], '-', False), (c[3], '-', False)]
    voting_thresholds = [0.1, 0.5, 0.9]

In [ ]:
media_db = load_media_db('../experimental/media/media_db.tsv')
seed_media = load_media_db('../experimental/media/kbase_media.tsv')

### Select models

In [ ]:
if organism == 'ecoli' and test == 'main':
    model_list = [
        ('iML1515', '../models/original/iML1515.xml', 'bigg'),
        ('CarveMe', '../models/CarveMe/Ecoli_K12_MG1655.xml', 'cobra'),
        ('modelSEED', '../models/modelSEED/ecoli.xml', 'seed')
    ]

if organism == 'bsubtilis' and test == 'main':
    model_list = [
        ('iYO844', '../models/original/iYO844.xml', 'bigg'),
        ('CarveMe', '../models/CarveMe/Bsubtilis_168.xml', 'cobra'),
        ('modelSEED', '../models/modelSEED/bsubtilis.xml', 'seed')
    ]

if organism == 'mgenitalium' and test == 'main':
    model_list = [
        ('iPS189', '../models/original/iPS189_fixed.xml', 'cobra'),
        ('CarveMe', '../models/CarveMe/M_genitalium_G37.xml', 'cobra'),
        ('modelSEED', '../models/modelSEED/mgenitalium.xml', 'seed')
    ]

if organism == 'paeruginosa' and test == 'main':
    model_list = [
        ('iMO1056', '../models/original/iMO1056_fixed.xml', 'cobra'),
        ('CarveMe', '../models/CarveMe/Paeruginosa_PAO1.xml', 'cobra'),
        ('modelSEED', '../models/modelSEED/paeruginosa.xml', 'seed')
    ]
    
if organism == 'soneidensis' and test == 'main':
        model_list = [
        ('iSO783', '../models/original/iSO783_fixed.xml', 'cobra'),
        ('CarveMe', '../models/CarveMe/Soneidensis_MR1.xml', 'cobra'),
        ('modelSEED', '../models/modelSEED/soneidensis.xml', 'seed')
    ]

In [ ]:
if organism == 'ecoli' and test == 'biomass':
    model_list = [
        ('iJO1366 biomass', '../models/CarveMe/Ecoli_K12_MG1655_iJO1366_biomass.xml', 'cobra'),
        ('Gram-neg biomass', '../models/CarveMe/Ecoli_K12_MG1655.xml', 'cobra'),
        ('Core biomass', '../models/CarveMe/Ecoli_K12_MG1655_core_biomass.xml', 'cobra'),
    ]

if organism == 'bsubtilis' and test == 'biomass':
    model_list = [
        ('iYO844 biomass', '../models/CarveMe/Bsubtilis_168_iYO844_biomass.xml', 'cobra'),
        ('Gram-pos biomass', '../models/CarveMe/Bsubtilis_168.xml', 'cobra'),
        ('Core biomass', '../models/CarveMe/Bsubtilis_168_core_biomass.xml', 'cobra'),
    ]

In [ ]:
if organism == 'ecoli' and test == 'ensemble':
    model_list = [
        ('Single', '../models/CarveMe/Ecoli_K12_MG1655.xml', 'cobra'),
        ('Ensemble (T={:.0%})', '../models/CarveMe/Ecoli_K12_MG1655_ensemble.xml', 'cobra')
    ]

if organism == 'bsubtilis' and test == 'ensemble':
    model_list = [
        ('Single', '../models/CarveMe/Bsubtilis_168.xml', 'cobra'),
        ('Ensemble (T={:.0%})', '../models/CarveMe/Bsubtilis_168_ensemble.xml', 'cobra')
    ]

if organism == 'paeruginosa' and test == 'ensemble':
    model_list = [
        ('Single', '../models/CarveMe/Paeruginosa_PAO1.xml', 'cobra'),
        ('Ensemble (T={:.0%})', '../models/CarveMe/Paeruginosa_PAO1_ensemble.xml', 'cobra')
    ]
    
if organism == 'soneidensis' and test == 'ensemble':
    model_list = [
        ('Single', '../models/CarveMe/Soneidensis_MR1.xml', 'cobra'),
        ('Ensemble (T={:.0%})', '../models/CarveMe/Soneidensis_MR1_ensemble.xml', 'cobra')
    ]
    
if organism == 'mgenitalium' and test == 'ensemble':
    model_list = [
        ('Single', '../models/CarveMe/M_genitalium_G37.xml', 'cobra'),
        ('Ensemble (T={:.0%})', '../models/CarveMe/M_genitalium_G37_ensemble.xml', 'cobra')
    ]

### Load models

In [ ]:
models = {}

for label, model_file, flavor in model_list: 
    if 'Ensemble' in label:
        models[label] = load_ensemble(model_file, flavor=flavor)
        models[label].size = 10

    else:
        models[label] = load_cbmodel(model_file, flavor=flavor)

### Run simulations

In [ ]:
if test == 'main':
    common_genes = None
    for label, _, _ in model_list: 
        if common_genes is None:
            common_genes = set(models[label].genes.keys())
        else:
            common_genes &= set(models[label].genes.keys())
    
    essential = essential & common_genes
    
    if non_essential:
        non_essential = non_essential & common_genes
    else:
        non_essential = common_genes - essential

In [ ]:
data_all = []
labels = []

for label, _, flavor in model_list: 
    
    if medium_tag is None:
        medium = None
    elif flavor == 'seed':
        medium = seed_media[medium_tag]
    else:
        medium = media_db[medium_tag] 
        
    ensemble = ('Ensemble' in label)
        
    res = benchmark_essentiality(models[label], medium, essential, non_essential,
                                 ensemble=ensemble,
                                 voting_thresholds=voting_thresholds,
                                 flavor=flavor)
    
    if ensemble:
        data_all.extend(res)
        labels.extend([label.format(t) for t in voting_thresholds])
    else:
        data_all.append(res)
        labels.append(label)

In [ ]:
spider_plot_compare(data_all, labels, None, styles=styles)
plt.savefig('../results/essentiality/{}_{}.png'.format(organism, test), dpi=300, bbox_inches='tight')